In [79]:
import numpy as np
import plotly.graph_objs as go
import autopeaks
from scipy.signal import lfilter
from edfplus import Edfplus
from plotly.offline import plot

In [16]:
%load_ext autoreload
%autoreload 2

# load ecg

In [4]:
f = open('../data/wsx_ecg.edf', 'rb')
raw = f.read()
f.close()

In [3]:
edf = Edfplus(raw)
ecg = edf.signals['ECG LL-RA']

In [10]:
ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.75,min_dist=300,fs=500)
list(map(ecg_auto_peaks.findpeaks, ecg));

In [11]:
ecg_peak_indices = ecg_auto_peaks.peak_indexes
ecg_peak_values = ecg_auto_peaks.peak_values

In [12]:
plot([go.Scatter(y=ecg), go.Scatter(x=ecg_peak_indices, y=ecg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

### find peaks in ecg

In [110]:
ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.73,min_dist=300,fs=500)
ecg_filter = lfilter(b,a,ecg)
list(map(ecg_auto_peaks.findpeaks, ecg_filter));

In [107]:
ecg_peak_indices = ecg_auto_peaks.peak_indexes
ecg_peak_values = ecg_auto_peaks.peak_values

In [111]:
plot([go.Scatter(y=ecg_filter), go.Scatter(x=ecg_peak_indices, y=ecg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

# load bcg

In [14]:
def _parse_integer(bytes):
    int_str = bytes.decode('ASCII')
    integers = []
    #try:
    #integers = list(map(int, int_str.split(',')))
    for s in int_str.split(','):
        try:
            integer = int(s)
        except:
            print(bytes)
            continue
        integers.append(integer)
    #except:
        #raise ValueError("")
    return integers
    
def parse_bcg(bytes):
    data = []
    packet = []
    start_index = 0
    end_index = 0
    for ind, byte in enumerate(bytes):
        if byte == ord('#'):
            if end_index - start_index > 0:
                #print(start_index, end_index)
                integers = _parse_integer(bytes[start_index:end_index])
                data.extend(integers)
            start_index = ind + 1
            end_index = ind + 1
        if byte == ord('\r'):
            end_index = ind
    if end_index - start_index > 0:
        integers = _parse_integer(bytes[start_index:end_index])
        data.extend(integers)
    return data

In [23]:
f = open("../data/wsx_bcg_wave(500HZ).txt", 'rb')
data = f.read()
f.close()
bcg = parse_bcg(data)
bcg = bcg[12726:]

### apply bandpass 0.5-20Hz filter

In [78]:
b = np.loadtxt("bcg_bandpass_b.csv", delimiter=',')
a = np.loadtxt("bcg_bandpass_a.csv", delimiter=',')

In [82]:
bcg_filter = lfilter(b, a, bcg)

In [47]:
start = 500*60*(5+15)
offset = 500*10
signal = bcg[start:start+offset]
ecg_part = ecg[start:start+offset]
##

In [146]:
bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.70,min_dist=290,fs=500)

In [147]:
%timeit list(map(bcg_auto_peaks.findpeaks, bcg_filter));

1.3 s ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [125]:
bcg_peak_indices = bcg_auto_peaks.peak_indexes
bcg_peak_values = bcg_auto_peaks.peak_values

In [121]:
plot([go.Scatter(y=ecg),go.Scatter(y=bcg_filter), go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

### compare heart interval betwwen bcg and ecfg

In [126]:
ecg_rr = np.diff(ecg_peak_indices)
bcg_jj = np.diff(bcg_peak_indices)
num_peaks = 100000
plot([go.Scatter(x=ecg_peak_indices[1:num_peaks+1],y=ecg_rr[:num_peaks], mode="lines+markers",name="ecg"), go.Scatter(x=np.array(bcg_peak_indices[1:num_peaks+1]),y=bcg_jj[:num_peaks],mode="lines+markers", name="bcg")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

In [128]:
example = bcg_jj[1000:1000+50]
example

array([ 392,  401,  401,  399,  403,  405,  404,  395,  396,  393,  389,
        381,  384,  386,  378,  377,  379,  382,  387,  382,  390,  400,
        403,  395,  400,  398,  389, 1641, 1015,  851,  450,  653,  311,
       1227,  411, 1000,  493,  701,  395,  299, 1134,  937, 1420,  343,
        621,  337,  993,  565,  418, 1027])

In [143]:
from cleanpeaks import best_jj_interval

In [144]:
best_jj_interval(example)

299.0 433.2


392.5

In [130]:
hist, bin_edges = np.histogram(example)

In [132]:
bin_edges,hist

(array([ 299. ,  433.2,  567.4,  701.6,  835.8,  970. , 1104.2, 1238.4,
        1372.6, 1506.8, 1641. ]),
 array([34,  3,  3,  0,  2,  4,  2,  0,  1,  1]))

In [145]:
pool = []
for peak in example:
    if (peak >= 299 and peak <= 433.2):
        pool.append(peak)
print(pool)
m = np.median(pool)
m 

[392, 401, 401, 399, 403, 405, 404, 395, 396, 393, 389, 381, 384, 386, 378, 377, 379, 382, 387, 382, 390, 400, 403, 395, 400, 398, 389, 311, 411, 395, 299, 343, 337, 418]


392.5

In [101]:
ecg_rr[1000:50]

array([ 527, 1723,  760,  580,  411,  600,  659,  386,  321,  324,  330,
        431,  479,  427,  313,  386,  923, 1061,  360,  561,  583,  547,
        685,  419,  511,  553,  602,  581,  344,  365,  319,  355,  571,
        305,  501,  390,  304,  319,  632,  310,  310,  319,  620,  311,
        310,  338, 1113,  354,  558,  306])

# Find Template

In [73]:
from algorithm.template import find_template, conv

In [74]:
template = find_template(signal, 200)
scores = conv(signal, template)

In [75]:
plot([go.Scatter(y=scores), go.Scatter(y=ecg_part)])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

In [77]:
plot([go.Scatter(y=signal), go.Scatter(y=ecg_part)])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

In [76]:
plot([go.Scatter(y=template)])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

# data visulation

In [83]:
plot([go.Scatter(y=signal,name="bcg"), ,
      go.Scatter(y=ecg[start:start+offset], name="ecg")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

In [85]:
plot([go.Scatter(y=ecg), go.Scatter(y=bcg), go.Scatter(y=bcg_filter,name="bcg_filter"),go.Scatter(x=ecg_peak_indices, y=ecg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'

In [89]:
plot([go.Scatter(y=bcg), go.Scatter(y=bcg_filter+300,name="bcg_filter")])

'file:///home/guo/Github/BCGHeart/notebook/temp-plot.html'